# 1. Use vnstock library to fetch stock information

In [1]:
from vnstock import Vnstock

stock = Vnstock().stock(symbol='TCB', source='TCBS')
stock.quote.history(start='2025-01-01', end='2025-11-28')

2025-11-28 21:02:25 - vnstock.common.data - INFO - TCBS listing data fallback to VCI


,time,open,high,low,close,volume
0,2024-11-28,23.04,23.09,22.80,22.99,8138330
1,2024-11-29,22.89,22.99,22.75,22.99,8449424
2,2024-12-02,23.09,23.09,22.80,23.04,6571784
3,2024-12-03,23.04,23.14,22.84,23.04,8334958
4,2024-12-04,22.84,22.99,22.80,22.84,8182047
...,...,...,...,...,...,...
245,2025-11-24,34.30,34.40,33.90,33.95,8285499
246,2025-11-25,34.00,34.00,33.45,33.45,9153171
247,2025-11-26,33.60,33.90,33.50,33.85,7865091
248,2025-11-27,33.95,34.20,33.70,33.75,8862611


In [2]:
df_stock = stock.quote.history(start='2025-01-01', end='2025-11-28')
df_stock

,time,open,high,low,close,volume
0,2024-11-28,23.04,23.09,22.80,22.99,8138330
1,2024-11-29,22.89,22.99,22.75,22.99,8449424
2,2024-12-02,23.09,23.09,22.80,23.04,6571784
3,2024-12-03,23.04,23.14,22.84,23.04,8334958
4,2024-12-04,22.84,22.99,22.80,22.84,8182047
...,...,...,...,...,...,...
245,2025-11-24,34.30,34.40,33.90,33.95,8285499
246,2025-11-25,34.00,34.00,33.45,33.45,9153171
247,2025-11-26,33.60,33.90,33.50,33.85,7865091
248,2025-11-27,33.95,34.20,33.70,33.75,8862611


In [3]:
df_stock.to_csv('2.stock_TCB_2025.csv')

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# 1. Đọc CSV
df = pd.read_csv(r'D:\Data Analyst VTI\5. Python\Stock TCB 2025\2.stock_TCB_2025.csv')

# 2. Chuẩn hóa connection string
params = urllib.parse.quote_plus(
    r"Driver={ODBC Driver 17 for SQL Server};"
    r"Server=localhost\SQLEXPRESS;"
    r"Database=stock_gemini_ai;"
    r"Trusted_Connection=yes;"
)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# 3. Push dataframe vào SQL Server
df.to_sql('stock_TCB_2025', con=engine, if_exists='replace', index=False)


250

# 2. Import Libary

In [5]:
import pandas as pd
import pyodbc

# 3. Import Data Frame From SQL SERVER

In [6]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=stock_gemini_ai;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df_stock = pd.read_sql_query('''
                                        
SELECT *
FROM [stock_gemini_ai].[dbo].[stock_TCB_2025]
                                       
    ; ''', conn)
df_stock

"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,Unnamed: 0,time,open,high,low,close,volume
0,0,2024-11-28,23.04,23.09,22.80,22.99,8138330
1,1,2024-11-29,22.89,22.99,22.75,22.99,8449424
2,2,2024-12-02,23.09,23.09,22.80,23.04,6571784
3,3,2024-12-03,23.04,23.14,22.84,23.04,8334958
4,4,2024-12-04,22.84,22.99,22.80,22.84,8182047
...,...,...,...,...,...,...,...
245,245,2025-11-24,34.30,34.40,33.90,33.95,8285499
246,246,2025-11-25,34.00,34.00,33.45,33.45,9153171
247,247,2025-11-26,33.60,33.90,33.50,33.85,7865091
248,248,2025-11-27,33.95,34.20,33.70,33.75,8862611


In [7]:
df_stock.drop('Unnamed: 0',axis=1, inplace=True)

# 4. Cleaning Data Using Detect Outlier Technical


## 4.1 Using IQR (Interquartile Range)

In [13]:
Q1 = df_stock['close'].quantile(0.25)
Q3 = df_stock['close'].quantile(0.75)
IQR = Q3 - Q1

outliers_iqr = df_stock[(df_stock['close'] < Q1 - 1.5*IQR) |
                        (df_stock['close'] > Q3 + 1.5*IQR)]

if outliers_iqr.empty:
    print("No outliers found")
else:
    print("Close price outliers are:", outliers_iqr)


No outliers found


## 4.2 Using Z-Score

In [12]:
import pandas as pd

mean_close = df_stock['close'].mean()
std_close = df_stock['close'].std()

# Tính z_score
df_stock['z_score'] = (df_stock['close'] - mean_close) / std_close

# Lọc outlier |Z| > 3
outliers_z = df_stock[df_stock['z_score'].abs() > 3]

# Nếu không có outlier
if outliers_z.empty:
    print("No outliers found")
else:
    print("Close price outliers are:", outliers_z)


No outliers found


In [10]:
import pandas as pd

# -------------------------------
# 1. Z-score
# -------------------------------
mean_close = df_stock['close'].mean()
std_close = df_stock['close'].std()

df_stock['z_score'] = (df_stock['close'] - mean_close) / std_close
df_stock['z_upper'] = mean_close + 3 * std_close
df_stock['z_lower'] = mean_close - 3 * std_close

# -------------------------------
# 2. IQR
# -------------------------------
Q1 = df_stock['close'].quantile(0.25)
Q3 = df_stock['close'].quantile(0.75)
IQR = Q3 - Q1

df_stock['iqr_upper'] = Q3 + 1.5 * IQR
df_stock['iqr_lower'] = Q1 - 1.5 * IQR

# -------------------------------
# 3. Xem kết quả
# -------------------------------
print(df_stock.head())


         time   open   high    low  close   volume   z_score    z_upper  \
0  2024-11-28  23.04  23.09  22.80  22.99  8138330 -1.359484  47.394673   
1  2024-11-29  22.89  22.99  22.75  22.99  8449424 -1.359484  47.394673   
2  2024-12-02  23.09  23.09  22.80  23.04  6571784 -1.350553  47.394673   
3  2024-12-03  23.04  23.14  22.84  23.04  8334958 -1.350553  47.394673   
4  2024-12-04  22.84  22.99  22.80  22.84  8182047 -1.386279  47.394673   

     z_lower  iqr_upper  iqr_lower  
0  13.806287   49.84875   10.61875  
1  13.806287   49.84875   10.61875  
2  13.806287   49.84875   10.61875  
3  13.806287   49.84875   10.61875  
4  13.806287   49.84875   10.61875  


# 5. Push data to SQL Server

In [11]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=stock_gemini_ai;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "stock_TCB_2025_outlier"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_stock[c].dtype)}" for c in df_stock.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_stock.columns))})", df_stock.values.tolist())
conn.commit()

"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
"\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
